In [1]:
%pip install transformers
%pip install PyPDF2

     |████████████████████████████████| 481kB 2.8MB/s 
     |████████████████████████████████| 870kB 56.9MB/s 
     |████████████████████████████████| 3.1MB 35.1MB/s 
     |████████████████████████████████| 1.0MB 46.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=75d2cb419c0985517a9fd750f3d971d36c028a0b5d3c36e64c1fdf63e3e67f05
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 81kB 2.6MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=75c524e069761cab07572aa8123703f15153cb529c38ca14406ef9681c1b8406
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [0]:
%rm -rf BERT_financialReportAnalysis/

In [3]:
!git clone https://github.com/francisliu2/BERT_financialReportAnalysis.git

Cloning into 'BERT_financialReportAnalysis'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 93 (delta 57), reused 59 (delta 23), pack-reused 0
Unpacking objects: 100% (93/93), done.


In [4]:
%cd BERT_financialReportAnalysis/

/content/BERT_financialReportAnalysis


In [5]:
from companytoVector_BERT import *
from corporate_profile_extrator import *

from google.colab import files
import os
import glob

import pandas as pd

os.mkdir('outputs')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



Max number of tokens in one sentence: 512


In [0]:
CP = glob.glob("./data/*.pdf")

In [7]:
model = load_model()

In [0]:
success = []
failed = []
result_temp = [] # for debugging
for i in range(len(CP)):
  name = CP[i].replace('.pdf', '') 
  name = name.replace('./data/', '')
  corporate_profile_text = get_corporate_profile(CP[i])
  corporate_profile_tokens = text_tokening(corporate_profile_text, verbose=1)
  corporate_profile_ids = corporate_profile_tokens[0]
  if corporate_profile_ids == None:
    failed.append({name, 'sentence is too long'})
  else:
    success.append(name)
    result = company2vector(corporate_profile_ids, model, average_over_sent=False)
    result_temp.append(result)
  
  pd_temp = vector_of_each_whole_token(corporate_profile_tokens, result)
  pd_temp.to_pickle('./outputs/'+name+'.pkl')
    # np.save('./outputs/'+name, result)

In [0]:
test = pd.read_pickle('./outputs/'+name+'.pkl')

In [12]:
len(pd_temp)

19

In [14]:
result.shape

(10, 512, 768)

In [0]:
success

['Yongnam Annual Report 2010', 'China Star_Sustainability Report']

In [0]:
!zip -r ./outputs.zip ./outputs
!sleep 2
files.download('outputs.zip')

In [0]:
files.download('outputs.zip')

In [0]:
# Check time needed to run
corporate_profile_text = get_corporate_profile(CP[0])
%time result = company2vector(corporate_profile_text, model)

CPU times: user 29.9 s, sys: 2.69 s, total: 32.6 s
Wall time: 32.7 s
